In [2]:
import numpy as np
import pandas as pd
import re
import nltk
import sklearn
import warnings
from platform import python_version
from sklearn.model_selection import train_test_split
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from sklearn.linear_model import Perceptron
from sklearn.svm import LinearSVC
print(python_version())

3.9.5


### Read Data:

In [3]:
# Could not load it directly from the url:
ratings_df = pd.read_csv("./amazon_reviews_us_Kitchen_v1_00.tsv", sep="\t",
                         error_bad_lines=False, warn_bad_lines=False)

In [4]:
ratings_df.head()

,marketplace,customer_id,review_id,product_id,product_parent,product_title,product_category,star_rating,helpful_votes,total_votes,vine,verified_purchase,review_headline,review_body,review_date
0,US,37000337,R3DT59XH7HXR9K,B00303FI0G,529320574,Arthur Court Paper Towel Holder,Kitchen,5.0,0.0,0.0,N,Y,Beautiful. Looks great on counter,Beautiful. Looks great on counter.,2015-08-31
1,US,15272914,R1LFS11BNASSU8,B00JCZKZN6,274237558,Olde Thompson Bavaria Glass Salt and Pepper Mi...,Kitchen,5.0,0.0,1.0,N,Y,Awesome & Self-ness,I personally have 5 days sets and have also bo...,2015-08-31
2,US,36137863,R296RT05AG0AF6,B00JLIKA5C,544675303,Progressive International PL8 Professional Man...,Kitchen,5.0,0.0,0.0,N,Y,Fabulous and worth every penny,Fabulous and worth every penny. Used for clean...,2015-08-31
3,US,43311049,R3V37XDZ7ZCI3L,B000GBNB8G,491599489,Zyliss Jumbo Garlic Press,Kitchen,5.0,0.0,1.0,N,Y,Five Stars,A must if you love garlic on tomato marinara s...,2015-08-31
4,US,13763148,R14GU232NQFYX2,B00VJ5KX9S,353790155,"1 X Premier Pizza Cutter - Stainless Steel 14""...",Kitchen,5.0,0.0,0.0,N,Y,Better than sex,Worth every penny! Buy one now and be a pizza ...,2015-08-31


In [5]:
# Simplify Dataset: Ensure all have reviews
ratings_df = ratings_df.loc[:, ["review_body", "star_rating"]]
ratings_df = ratings_df[ratings_df["review_body"].notnull()]
ratings_df.head()

,review_body,star_rating
0,Beautiful. Looks great on counter.,5.0
1,I personally have 5 days sets and have also bo...,5.0
2,Fabulous and worth every penny. Used for clean...,5.0
3,A must if you love garlic on tomato marinara s...,5.0
4,Worth every penny! Buy one now and be a pizza ...,5.0


In [6]:
# Gather 50k of each rating through random selection:
rating_1 = ratings_df[ratings_df["star_rating"] == 1].sample(50000)
rating_2 = ratings_df[ratings_df["star_rating"] == 2].sample(50000)
rating_3 = ratings_df[ratings_df["star_rating"] == 3].sample(50000)
rating_4 = ratings_df[ratings_df["star_rating"] == 4].sample(50000)
rating_5 = ratings_df[ratings_df["star_rating"] == 5].sample(50000)

ratings_sampled_df = pd.concat([rating_1, rating_2, rating_3, rating_4, rating_5])
ratings_sampled_df = ratings_sampled_df.sample(frac=1)
ratings_sampled_df.reset_index(drop=True, inplace=True)
ratings_sampled_df.head()

,review_body,star_rating
0,BUYER BEWARE! Absolute waste of money. Might a...,1.0
1,Woks great This is our first dehydrator and ev...,4.0
2,"do not like this maker at all, hard to use , m...",1.0
3,this list of qualities on the main product pag...,1.0
4,"Very nice kitchen tool, Seems to me made of Ba...",4.0


In [7]:
# Map star rating to sentiment rating: We will assign -1 as neutral
d_ = {4:1, 5:1, 1:0, 2:0, 3:2}
ratings_sampled_df["sentiment"] = ratings_sampled_df["star_rating"].map(d_)
ratings_sampled_df.head()

,review_body,star_rating,sentiment
0,BUYER BEWARE! Absolute waste of money. Might a...,1.0,0
1,Woks great This is our first dehydrator and ev...,4.0,1
2,"do not like this maker at all, hard to use , m...",1.0,0
3,this list of qualities on the main product pag...,1.0,0
4,"Very nice kitchen tool, Seems to me made of Ba...",4.0,1


In [8]:
# Train-test Split
train, test = train_test_split(ratings_sampled_df, test_size=0.2)
train.reset_index(drop=True, inplace=True)
test.reset_index(drop=True, inplace=True)

### Word Embedding:

In [9]:
# Load in the word2vec:
import gensim
word2vec = gensim.models.KeyedVectors.load_word2vec_format('./GoogleNews-vectors-negative300.bin', binary=True)

In [10]:
#Check Smeantic Similarities: (close enough!)
result_vec = word2vec["king"] - word2vec["man"] + word2vec["woman"]
word2vec.most_similar(positive=result_vec, topn=3)

[('king', 0.844939112663269),
 ('queen', 0.7300516366958618),
 ('monarch', 0.6454660296440125)]

In [11]:
word2vec.most_similar(positive="excellent", topn=3)

[('terrific', 0.7409728765487671),
 ('superb', 0.7062715888023376),
 ('exceptional', 0.681470513343811)]

In [12]:
# Train a Word2Vec model using own dataset: embedding_size=300, window_size=11, min_word_cnt=10
# Clean data:
train["review_body"] = train["review_body"].str.lower()
test["review_body"] = test["review_body"].str.lower()

train["review_body"] = train["review_body"].replace(r'http\S+|www.\S+', '', regex=True)
test["review_body"] = test["review_body"].replace(r'http\S+|www.\S+', '', regex=True)


train["review_body"] = train["review_body"].replace(r'[^a-z|\s]', '', regex=True)
test["review_body"] = test["review_body"].replace(r'[^a-z|\s]', '', regex=True)

train["review_body"] = train["review_body"].replace(r'\s\s+', ' ', regex=True)
test["review_body"] = test["review_body"].replace(r'\s\s+', ' ', regex=True)

<ipython-input-12-2b2b707881ec>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train["review_body"] = train["review_body"].str.lower()
<ipython-input-12-2b2b707881ec>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test["review_body"] = test["review_body"].str.lower()
<ipython-input-12-2b2b707881ec>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pyd

In [13]:
stop_words = set(stopwords.words('english'))

def remove_stopwords(s):
    tokens = word_tokenize(s)
    filtered_tokens = [word for word in tokens if word not in stop_words]
    return filtered_tokens

train["review_body"] = train["review_body"].apply(remove_stopwords)
test["review_body"] = test["review_body"].apply(remove_stopwords)

<ipython-input-13-7e488c7e9935>:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train["review_body"] = train["review_body"].apply(remove_stopwords)
<ipython-input-13-7e488c7e9935>:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test["review_body"] = test["review_body"].apply(remove_stopwords)


In [14]:
train_sentences = np.array(train["review_body"])
test_sentences = np.array(test["review_body"])

In [15]:
model = gensim.models.Word2Vec(train_sentences, min_count=10, vector_size=200, window=11)

In [16]:
# Test semantic similarities:
result_vec = model.wv["king"] - model.wv["man"] + model.wv["woman"]
model.wv.similar_by_vector(result_vec)

[('king', 0.6640127301216125),
 ('norpro', 0.49288639426231384),
 ('whereas', 0.4772895276546478),
 ('stainlesssteel', 0.4654127359390259),
 ('triply', 0.44628649950027466),
 ('kirkland', 0.4440610110759735),
 ('frypan', 0.4382425546646118),
 ('rimmed', 0.42901623249053955),
 ('winco', 0.42809468507766724),
 ('qt', 0.42460620403289795)]

In [17]:
model.wv.similar_by_word("excellent")

[('outstanding', 0.7735501527786255),
 ('fantastic', 0.7434232831001282),
 ('exceptional', 0.7344864010810852),
 ('wonderful', 0.6633721590042114),
 ('terrific', 0.6460570693016052),
 ('superb', 0.643336832523346),
 ('great', 0.6258797645568848),
 ('pleased', 0.5849264860153198),
 ('amazing', 0.569335401058197),
 ('satisfactory', 0.5572806596755981)]

From the Semantic Similarities above we can see that our model actually performs relatively well given the results of the top 10 closest words to Excellent. However we can see the problem emerges that the reviews do not necessarily contain all the words we might want such as "Queen", which may or may not be in the data based on the randomness involved in selecting reviews. One would prefer to utilize the all encompassing Google word2vec to help with unorthodox words, and we can trust that the performance is better. Yet, the create w2v does produce results faster.

### Simple Models:

#### Personally Trained Word2Vec

In [18]:
# Need to make binary model at this point:
train_binary = train[~(train["sentiment"] == 2)]
test_binary = test[~(test["sentiment"] == 2)]

In [19]:
X_train = train_binary["review_body"]
Y_train = train_binary["sentiment"].values

X_test = test_binary["review_body"]
Y_test = test_binary["sentiment"].values

In [65]:
def avg_vector_mine(review):
    tot_ = 0
    for word_ in review:
        if word_ in model.wv:
            tot_ += model.wv[word_]
        else:
            continue
    if len(review) == 0:
        return 0
    return tot_/len(review)

In [66]:
# Vectorize X_train:
X_train_avg = X_train.apply(avg_vector_mine)
X_train_df = pd.DataFrame.from_dict(dict(zip(X_train_avg.index, X_train_avg.values))).T
X_train_vectorized_mine = X_train_df.values

In [67]:
# Vectorize X_test:
X_test_avg = X_test.apply(avg_vector_mine)
X_test_df = pd.DataFrame.from_dict(dict(zip(X_test_avg.index, X_test_avg.values))).T
X_test_vectorized_mine = X_test_df.values

In [68]:
perceptron = Perceptron(tol=1e-3, random_state=42)
perceptron.fit(X_train_vectorized_mine, Y_train)
print("Perceptron Training Accuracy:", perceptron.score(X_train_vectorized_mine, Y_train))

Perceptron Training Accuracy: 0.7884838866882428


In [69]:
print("Perceptron Test Accuracy:", perceptron.score(X_test_vectorized_mine, Y_test))

Perceptron Test Accuracy: 0.7883893255895459


In [70]:
svm = LinearSVC(max_iter=1000)
svm.fit(X_train_vectorized_mine, Y_train)
print("SVM Training Accuracy:", svm.score(X_train_vectorized_mine, Y_train))

SVM Training Accuracy: 0.8589850925312706


/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/sklearn/svm/_base.py:985: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn("Liblinear failed to converge, increase "


In [71]:
print("SVM Test Accuracy:", svm.score(X_test_vectorized_mine, Y_test))

SVM Test Accuracy: 0.8611375356731588


#### Pretrained Word2Vec

In [72]:
def avg_vector_google(review):
    tot_ = 0
    for word_ in review:
        if word_ in word2vec:
            tot_ += word2vec[word_]
        else:
            continue
    if len(review) == 0:
        return 0
    return tot_/len(review)

In [73]:
# Vectorize X_train:
X_train_avg = X_train.apply(avg_vector_google)
X_train_df = pd.DataFrame.from_dict(dict(zip(X_train_avg.index, X_train_avg.values))).T
X_train_vectorized_google = X_train_df.values

In [74]:
# Vectorize X_test:
X_test_avg = X_test.apply(avg_vector_google)
X_test_df = pd.DataFrame.from_dict(dict(zip(X_test_avg.index, X_test_avg.values))).T
X_test_vectorized_google = X_test_df.values

In [75]:
perceptron = Perceptron(tol=1e-3, random_state=42)
perceptron.fit(X_train_vectorized_google, Y_train)
print("Perceptron Training Accuracy:", perceptron.score(X_train_vectorized_google, Y_train))

Perceptron Training Accuracy: 0.7255363814712534


In [76]:
print("Perceptron Test Accuracy:", perceptron.score(X_test_vectorized_google, Y_test))

Perceptron Test Accuracy: 0.7252290592299604


In [77]:
svm = LinearSVC(max_iter=1000)
svm.fit(X_train_vectorized_google, Y_train)
print("SVM Training Accuracy:", svm.score(X_train_vectorized_google, Y_train))

SVM Training Accuracy: 0.8195109150661651


In [78]:
print("SVM Test Accuracy:", svm.score(X_test_vectorized_google, Y_test))

SVM Test Accuracy: 0.8184298803384569


Perceptron Results TF.IDF: Test-Accuracy = .8569 <br>
SVM Results TF.IDF: Test-Accuracy = 0.898025 <br>
<br>
We can see from the above results that the models are better for TF.IDF, then personally trained Word2Vec, and lastly for the Google pre-trained Word2Vec. One likely explanation for this could be that the TF.IDF and personally train Word2Vec might be more tailored to the specific review set at hand. In addition, the averaging of the word vectors might be a very inefficient way to combine words in a review.

### Feedforward Neural Networks

#### Avg Word2Vec Vectors - Personally Trained Word2Vec:

In [20]:
import torch
import torch.nn.functional as F
from torch.utils.data import DataLoader
from torch.utils.data import TensorDataset
from torch import nn

In [80]:
class MLP(nn.Module):

    def __init__(self):
        super().__init__()

        input_dim = 200
        hidden_1 = 50
        hidden_2 = 10
        
        self.fc1 = nn.Linear(input_dim, hidden_1)
        self.fc2 = nn.Linear(hidden_1, hidden_2)
        self.fc3 = nn.Linear(hidden_2, 2)
        self.dropout = nn.Dropout(0.2)
        #self.act3 = nn.Softmax(dim=1)

    def forward(self, x):
        x = x.view(-1, 200)
        x = F.relu(self.fc1(x))
        x = self.dropout(x)
        x = F.relu(self.fc2(x))
        x = self.dropout(x)
        x = self.fc3(x)
        return x

In [81]:
# Formatting the data for DataLoader (coverts to tensors):
train_data = []
for i in range(len(X_train_vectorized_mine)):
    train_data.append([X_train_vectorized_mine[i], Y_train[i]])
    
test_data = []
for i in range(len(X_test_vectorized_mine)):
    test_data.append([X_test_vectorized_mine[i], Y_test[i]])

In [82]:
n_epochs = 10
trainloader = torch.utils.data.DataLoader(train_data, batch_size=20, num_workers=1)
testloader = torch.utils.data.DataLoader(test_data, batch_size=20, num_workers=1)
mlp = MLP()
criterion = nn.CrossEntropyLoss()
# optimizer = torch.optim.Adam(mlp.parameters(), lr=1e-4)
optimizer = torch.optim.SGD(mlp.parameters(), lr=0.01)

test_loss_min = 5

for epoch in range(n_epochs):
    train_loss = 0
    test_loss = 0
    
    mlp.train()
    for data, target in trainloader:
        # Zero the gradients
        optimizer.zero_grad()

        # Perform forward pass
        output = mlp(data.float())

        # Compute loss
        loss = criterion(output, target)

        # Perform backward pass
        loss.backward()

        # Perform optimization
        optimizer.step()

        # Print statistics
        train_loss += loss.item()*data.size(0)
        
    with torch.no_grad():
        for data, target in testloader:
            output = mlp(data.float())
            loss = criterion(output, target)
            test_loss += loss.item()*data.size(0)
        
    train_loss = train_loss/len(trainloader.dataset)
    test_loss = test_loss/len(testloader.dataset)
    
    print('Epoch: {} \tTraining Loss: {:.6f} \tTest Loss: {:.6f}'.format(
        epoch+1, 
        train_loss,
        test_loss
        ))
    
    if test_loss <= test_loss_min:
        torch.save(mlp.state_dict(), 'model.pt')
        test_loss_min = test_loss

  # Process is complete.
print('All done.')

Epoch: 1 	Training Loss: 0.401960 	Test Loss: 0.347470
Epoch: 2 	Training Loss: 0.346261 	Test Loss: 0.339061
Epoch: 3 	Training Loss: 0.336808 	Test Loss: 0.330312
Epoch: 4 	Training Loss: 0.331495 	Test Loss: 0.325287
Epoch: 5 	Training Loss: 0.324839 	Test Loss: 0.324928
Epoch: 6 	Training Loss: 0.322088 	Test Loss: 0.321494
Epoch: 7 	Training Loss: 0.318378 	Test Loss: 0.322282
Epoch: 8 	Training Loss: 0.317127 	Test Loss: 0.318651
Epoch: 9 	Training Loss: 0.314721 	Test Loss: 0.319576
Epoch: 10 	Training Loss: 0.313004 	Test Loss: 0.317683
All done.


In [83]:
mlp.load_state_dict(torch.load('model.pt'))
testloader = torch.utils.data.DataLoader(test_data, batch_size=1, num_workers=1)

In [21]:
# Calculate Accuracy from trained model:
def predict(model, dataloader):
    prediction_list = []
    with torch.no_grad():
        for data, target in dataloader:
            output = model(data.float())
            _, predicted = torch.max(output.data, 1) 
            prediction_list.append(predicted)
    return prediction_list

In [85]:
predictions = predict(mlp, testloader)
predictions = np.array(predictions)

In [22]:
def accuracy(y_true, y_pred):
    score = sum(y_true == y_pred)/len(y_pred)
    return score

In [87]:
print("Test Accuracy:", accuracy(Y_test, predictions))

Test Accuracy: 0.8694988234116057


In [23]:
# Formatting process for 3 class:
X_train_3 = train["review_body"]
Y_train_3 = train["sentiment"].values

X_test_3 = test["review_body"]
Y_test_3 = test["sentiment"].values

In [89]:
# Vectorize X_train_3:
X_train_avg_3 = X_train_3.apply(avg_vector_mine)
X_train_df_3 = pd.DataFrame.from_dict(dict(zip(X_train_avg_3.index, X_train_avg_3.values))).T
X_train_vectorized_mine_3 = X_train_df_3.values

In [90]:
# Vectorize X_test_3:
X_test_avg_3 = X_test_3.apply(avg_vector_mine)
X_test_df_3 = pd.DataFrame.from_dict(dict(zip(X_test_avg_3.index, X_test_avg_3.values))).T
X_test_vectorized_mine_3 = X_test_df_3.values

In [91]:
# Formatting the data for DataLoader (coverts to tensors):
train_data_3 = []
for i in range(len(X_train_vectorized_mine_3)):
    train_data_3.append([X_train_vectorized_mine_3[i], Y_train_3[i]])
    
test_data_3 = []
for i in range(len(X_test_vectorized_mine_3)):
    test_data_3.append([X_test_vectorized_mine_3[i], Y_test_3[i]])

In [92]:
class MLP_3(nn.Module):

    def __init__(self):
        super().__init__()

        input_dim = 200
        hidden_1 = 50
        hidden_2 = 10
        
        self.fc1 = nn.Linear(input_dim, hidden_1)
        self.fc2 = nn.Linear(hidden_1, hidden_2)
        self.fc3 = nn.Linear(hidden_2, 3)
        self.dropout = nn.Dropout(0.2)

    def forward(self, x):
        x = x.view(-1, 200)
        x = F.relu(self.fc1(x))
        x = self.dropout(x)
        x = F.relu(self.fc2(x))
        x = self.dropout(x)
        x = self.fc3(x)
        return x

In [93]:
n_epochs = 10
trainloader = torch.utils.data.DataLoader(train_data_3, batch_size=20, num_workers=1)
testloader = torch.utils.data.DataLoader(test_data_3, batch_size=20, num_workers=1)
mlp_3 = MLP_3()
criterion = nn.CrossEntropyLoss()
#optimizer = torch.optim.Adam(mlp.parameters(), lr=.0001)
optimizer = torch.optim.SGD(mlp_3.parameters(), lr=0.01)

test_loss_min = 5

for epoch in range(n_epochs):
    train_loss = 0
    test_loss = 0
    
    mlp_3.train()
    for data, target in trainloader:
        # Zero the gradients
        optimizer.zero_grad()

        # Perform forward pass
        output = mlp_3(data.float())

        # Compute loss
        loss = criterion(output, target)

        # Perform backward pass
        loss.backward()

        # Perform optimization
        optimizer.step()

        # Print statistics
        train_loss += loss.item()*data.size(0)
        
    with torch.no_grad():
        for data, target in testloader:
            output = mlp_3(data.float())
            loss = criterion(output, target)
            test_loss += loss.item()*data.size(0)
        
    train_loss = train_loss/len(trainloader.dataset)
    test_loss = test_loss/len(testloader.dataset)
    
    print('Epoch: {} \tTraining Loss: {:.6f} \tTest Loss: {:.6f}'.format(
        epoch+1, 
        train_loss,
        test_loss
        ))
    
    if test_loss <= test_loss_min:
        torch.save(mlp_3.state_dict(), 'model_3.pt')
        test_loss_min = test_loss

  # Process is complete.
print('All done.')

Epoch: 1 	Training Loss: 0.799257 	Test Loss: 0.753471
Epoch: 2 	Training Loss: 0.744144 	Test Loss: 0.733876
Epoch: 3 	Training Loss: 0.730217 	Test Loss: 0.725554
Epoch: 4 	Training Loss: 0.722700 	Test Loss: 0.722737
Epoch: 5 	Training Loss: 0.718984 	Test Loss: 0.719500
Epoch: 6 	Training Loss: 0.715308 	Test Loss: 0.715966
Epoch: 7 	Training Loss: 0.714287 	Test Loss: 0.715359
Epoch: 8 	Training Loss: 0.710467 	Test Loss: 0.712566
Epoch: 9 	Training Loss: 0.707925 	Test Loss: 0.714819
Epoch: 10 	Training Loss: 0.705990 	Test Loss: 0.710956
All done.


In [94]:
mlp_3.load_state_dict(torch.load('model_3.pt'))
testloader = torch.utils.data.DataLoader(test_data_3, batch_size=1, num_workers=1)

In [95]:
predictions = predict(mlp_3, testloader)
predictions = np.array(predictions)

In [96]:
print("Test Accuracy:", accuracy(Y_test_3, predictions))

Test Accuracy: 0.70034


#### Avg Word2Vec Vectors - Google Trained Word2Vec:

In [97]:
# Formatting the data for DataLoader (coverts to tensors):
train_data = []
for i in range(len(X_train_vectorized_google)):
    train_data.append([X_train_vectorized_google[i], Y_train[i]])
    
test_data = []
for i in range(len(X_test_vectorized_google)):
    test_data.append([X_test_vectorized_google[i], Y_test[i]])

In [98]:
class MLP_Google(nn.Module):

    def __init__(self):
        super().__init__()

        input_dim = 300
        hidden_1 = 50
        hidden_2 = 10
        
        self.fc1 = nn.Linear(input_dim, hidden_1)
        self.fc2 = nn.Linear(hidden_1, hidden_2)
        self.fc3 = nn.Linear(hidden_2, 2)
        self.dropout = nn.Dropout(0.2)
        #self.act3 = nn.Softmax(dim=1)

    def forward(self, x):
        x = x.view(-1, 300)
        x = F.relu(self.fc1(x))
        x = self.dropout(x)
        x = F.relu(self.fc2(x))
        x = self.dropout(x)
        x = self.fc3(x)
        return x

In [99]:
n_epochs = 10
trainloader = torch.utils.data.DataLoader(train_data, batch_size=20, num_workers=1)
testloader = torch.utils.data.DataLoader(test_data, batch_size=20, num_workers=1)
mlp_g = MLP_Google()
criterion = nn.CrossEntropyLoss()
# optimizer = torch.optim.Adam(mlp_g.parameters(), lr=1e-4)
optimizer = torch.optim.SGD(mlp_g.parameters(), lr=0.01)

test_loss_min = 5

for epoch in range(n_epochs):
    train_loss = 0
    test_loss = 0
    
    mlp_g.train()
    for data, target in trainloader:
        # Zero the gradients
        optimizer.zero_grad()

        # Perform forward pass
        output = mlp_g(data.float())

        # Compute loss
        loss = criterion(output, target)

        # Perform backward pass
        loss.backward()

        # Perform optimization
        optimizer.step()

        # Print statistics
        train_loss += loss.item()*data.size(0)
        
    with torch.no_grad():
        for data, target in testloader:
            output = mlp_g(data.float())
            loss = criterion(output, target)
            test_loss += loss.item()*data.size(0)
        
    train_loss = train_loss/len(trainloader.dataset)
    test_loss = test_loss/len(testloader.dataset)
    
    print('Epoch: {} \tTraining Loss: {:.6f} \tTest Loss: {:.6f}'.format(
        epoch+1, 
        train_loss,
        test_loss
        ))
    
    if test_loss <= test_loss_min:
        torch.save(mlp_g.state_dict(), 'model_google.pt')
        test_loss_min = test_loss

  # Process is complete.
print('All done.')

Epoch: 1 	Training Loss: 0.602880 	Test Loss: 0.480752
Epoch: 2 	Training Loss: 0.456435 	Test Loss: 0.436863
Epoch: 3 	Training Loss: 0.432322 	Test Loss: 0.423295
Epoch: 4 	Training Loss: 0.421360 	Test Loss: 0.416590
Epoch: 5 	Training Loss: 0.416135 	Test Loss: 0.410527
Epoch: 6 	Training Loss: 0.410357 	Test Loss: 0.408290
Epoch: 7 	Training Loss: 0.406344 	Test Loss: 0.404594
Epoch: 8 	Training Loss: 0.402517 	Test Loss: 0.401754
Epoch: 9 	Training Loss: 0.400275 	Test Loss: 0.400827
Epoch: 10 	Training Loss: 0.396903 	Test Loss: 0.399124
All done.


In [100]:
mlp_g.load_state_dict(torch.load('model_google.pt'))
testloader = torch.utils.data.DataLoader(test_data, batch_size=1, num_workers=1)

In [101]:
predictions = predict(mlp_g, testloader)
predictions = np.array(predictions)

In [102]:
print("Test Accuracy:", accuracy(Y_test, predictions))

Test Accuracy: 0.8293696490261854


In [103]:
# Vectorize X_train_3:
X_train_avg_3 = X_train_3.apply(avg_vector_google)
X_train_df_3 = pd.DataFrame.from_dict(dict(zip(X_train_avg_3.index, X_train_avg_3.values))).T
X_train_vectorized_google_3 = X_train_df_3.values

In [104]:
# Vectorize X_test_3:
X_test_avg_3 = X_test_3.apply(avg_vector_google)
X_test_df_3 = pd.DataFrame.from_dict(dict(zip(X_test_avg_3.index, X_test_avg_3.values))).T
X_test_vectorized_google_3 = X_test_df_3.values

In [105]:
# Formatting the data for DataLoader (coverts to tensors):
train_data = []
for i in range(len(X_train_vectorized_google_3)):
    train_data.append([X_train_vectorized_google_3[i], Y_train_3[i]])
    
test_data = []
for i in range(len(X_test_vectorized_google_3)):
    test_data.append([X_test_vectorized_google_3[i], Y_test_3[i]])

In [106]:
class MLP_Google_3(nn.Module):

    def __init__(self):
        super().__init__()

        input_dim = 300
        hidden_1 = 50
        hidden_2 = 10
        
        self.fc1 = nn.Linear(input_dim, hidden_1)
        self.fc2 = nn.Linear(hidden_1, hidden_2)
        self.fc3 = nn.Linear(hidden_2, 3)
        self.dropout = nn.Dropout(0.2)

    def forward(self, x):
        x = x.view(-1, 300)
        x = F.relu(self.fc1(x))
        x = self.dropout(x)
        x = F.relu(self.fc2(x))
        x = self.dropout(x)
        x = self.fc3(x)
        return x

In [107]:
n_epochs = 10
trainloader = torch.utils.data.DataLoader(train_data, batch_size=20, num_workers=1)
testloader = torch.utils.data.DataLoader(test_data, batch_size=20, num_workers=1)
mlp_g_3 = MLP_Google_3()
criterion = nn.CrossEntropyLoss()
# optimizer = torch.optim.Adam(mlp_g.parameters(), lr=1e-4)
optimizer = torch.optim.SGD(mlp_g_3.parameters(), lr=0.01)

test_loss_min = 5

for epoch in range(n_epochs):
    train_loss = 0
    test_loss = 0
    
    mlp_g_3.train()
    for data, target in trainloader:
        # Zero the gradients
        optimizer.zero_grad()

        # Perform forward pass
        output = mlp_g_3(data.float())

        # Compute loss
        loss = criterion(output, target)

        # Perform backward pass
        loss.backward()

        # Perform optimization
        optimizer.step()

        # Print statistics
        train_loss += loss.item()*data.size(0)
        
    with torch.no_grad():
        for data, target in testloader:
            output = mlp_g_3(data.float())
            loss = criterion(output, target)
            test_loss += loss.item()*data.size(0)
        
    train_loss = train_loss/len(trainloader.dataset)
    test_loss = test_loss/len(testloader.dataset)
    
    print('Epoch: {} \tTraining Loss: {:.6f} \tTest Loss: {:.6f}'.format(
        epoch+1, 
        train_loss,
        test_loss
        ))
    
    if test_loss <= test_loss_min:
        torch.save(mlp_g_3.state_dict(), 'model_google_3.pt')
        test_loss_min = test_loss

  # Process is complete.
print('All done.')

Epoch: 1 	Training Loss: 0.995692 	Test Loss: 0.882435
Epoch: 2 	Training Loss: 0.855592 	Test Loss: 0.838903
Epoch: 3 	Training Loss: 0.832647 	Test Loss: 0.826379
Epoch: 4 	Training Loss: 0.825438 	Test Loss: 0.819853
Epoch: 5 	Training Loss: 0.819547 	Test Loss: 0.816651
Epoch: 6 	Training Loss: 0.815187 	Test Loss: 0.812054
Epoch: 7 	Training Loss: 0.813337 	Test Loss: 0.814574
Epoch: 8 	Training Loss: 0.810747 	Test Loss: 0.810736
Epoch: 9 	Training Loss: 0.807258 	Test Loss: 0.806242
Epoch: 10 	Training Loss: 0.804989 	Test Loss: 0.805657
All done.


In [108]:
mlp_g_3.load_state_dict(torch.load('model_google_3.pt'))
testloader = torch.utils.data.DataLoader(test_data, batch_size=1, num_workers=1)

In [109]:
predictions = predict(mlp_g_3, testloader)
predictions = np.array(predictions)

In [110]:
print("Test Accuracy:", accuracy(Y_test_3, predictions))

Test Accuracy: 0.65562


### Binary & Ternary Models For First 10 W2V Vectors Models:

In [111]:
def first_10_w2v_mine(review):
    features = np.zeros((10,200))
    for i, word in enumerate(review):
        if i >= 10:
            break
        elif word in model.wv:
            features[i] = model.wv[word]
        else:
            continue
    return features            

In [112]:
# Vectorize X_train, X_test:
X_train_10 = X_train.apply(first_10_w2v_mine)
X_test_10 = X_test.apply(first_10_w2v_mine)

In [113]:
# Formatting the data for DataLoader (coverts to tensors):
train_data = []
for i in range(len(X_train_10)):
    train_data.append([X_train_10.iloc[i], Y_train[i]])

In [114]:
test_data = []
for i in range(len(X_test_10)):
    test_data.append([X_test_10.iloc[i], Y_test[i]])

In [115]:
class MLP_Mine_Binary_10(nn.Module):

    def __init__(self):
        super().__init__()

        vocab_size = 10
        embedding_size = 200
        hidden_1 = 50
        hidden_2 = 10
        
#         self.embeddings = nn.Embedding(vocab_size, embedding_size)
        self.fc1 = nn.Linear(vocab_size*embedding_size, hidden_1)
        self.fc2 = nn.Linear(hidden_1, hidden_2)
        self.fc3 = nn.Linear(hidden_2, 2)
        self.dropout = nn.Dropout(0.2)

    def forward(self, x):
        x = x.view(-1, 2000)
        x = F.relu(self.fc1(x))
        x = self.dropout(x)
        x = F.relu(self.fc2(x))
        x = self.dropout(x)
        x = self.fc3(x)
        return x

In [116]:
n_epochs = 5
trainloader = torch.utils.data.DataLoader(train_data, batch_size=20, num_workers=1)
testloader = torch.utils.data.DataLoader(test_data, batch_size=20, num_workers=1)
mlp_mine_binary_10 = MLP_Mine_Binary_10()
criterion = nn.CrossEntropyLoss()
# optimizer = torch.optim.Adam(mlp_g.parameters(), lr=1e-4)
optimizer = torch.optim.SGD(mlp_mine_binary_10.parameters(), lr=0.01)

test_loss_min = 5

for epoch in range(n_epochs):
    train_loss = 0
    test_loss = 0
    
    mlp_mine_binary_10.train()
    for data, target in trainloader:
        # Zero the gradients
        optimizer.zero_grad()

        # Perform forward pass
        output = mlp_mine_binary_10(data.float())

        # Compute loss
        loss = criterion(output, target)

        # Perform backward pass
        loss.backward()

        # Perform optimization
        optimizer.step()

        # Print statistics
        train_loss += loss.item()*data.size(0)
        
    with torch.no_grad():
        for data, target in testloader:
            output = mlp_mine_binary_10(data.float())
            loss = criterion(output, target)
            test_loss += loss.item()*data.size(0)
        
    train_loss = train_loss/len(trainloader.dataset)
    test_loss = test_loss/len(testloader.dataset)
    
    print('Epoch: {} \tTraining Loss: {:.6f} \tTest Loss: {:.6f}'.format(
        epoch+1, 
        train_loss,
        test_loss
        ))
    
    if test_loss <= test_loss_min:
        torch.save(mlp_mine_binary_10.state_dict(), 'model_mine_binary_10.pt')
        test_loss_min = test_loss

  # Process is complete.
print('All done.')

Epoch: 1 	Training Loss: 0.477691 	Test Loss: 0.449491
Epoch: 2 	Training Loss: 0.431447 	Test Loss: 0.443489
Epoch: 3 	Training Loss: 0.415830 	Test Loss: 0.439789
Epoch: 4 	Training Loss: 0.402873 	Test Loss: 0.442143
Epoch: 5 	Training Loss: 0.389524 	Test Loss: 0.447257
All done.


In [117]:
mlp_mine_binary_10.load_state_dict(torch.load('model_mine_binary_10.pt'))
testloader = torch.utils.data.DataLoader(test_data, batch_size=1, num_workers=1)

In [118]:
predictions = predict(mlp_mine_binary_10, testloader)
predictions = np.array(predictions)

In [119]:
print("Test Accuracy:", accuracy(Y_test, predictions))

Test Accuracy: 0.7984028438391829


In [120]:
# Vectorize X_train_3, X_test_3:
X_train_3_10 = X_train_3.apply(first_10_w2v_mine)
X_test_3_10 = X_test_3.apply(first_10_w2v_mine)

In [121]:
# Formatting the data for DataLoader (coverts to tensors):
train_data_3 = []
for i in range(len(X_train_3_10)):
    train_data_3.append([X_train_3_10.iloc[i], Y_train_3[i]])
    
test_data_3 = []
for i in range(len(X_test_3_10 )):
    test_data_3.append([X_test_3_10.iloc[i], Y_test_3[i]])

In [122]:
class MLP_Mine_Ternary_10(nn.Module):

    def __init__(self):
        super().__init__()

        vocab_size = 10
        embedding_size = 200
        hidden_1 = 50
        hidden_2 = 10
        
#         self.embeddings = nn.Embedding(vocab_size, embedding_size)
        self.fc1 = nn.Linear(vocab_size*embedding_size, hidden_1)
        self.fc2 = nn.Linear(hidden_1, hidden_2)
        self.fc3 = nn.Linear(hidden_2, 3)
        self.dropout = nn.Dropout(0.2)

    def forward(self, x):
        x = x.view(-1, 2000)
        x = F.relu(self.fc1(x))
        x = self.dropout(x)
        x = F.relu(self.fc2(x))
        x = self.dropout(x)
        x = self.fc3(x)
        return x

In [123]:
n_epochs = 5
trainloader = torch.utils.data.DataLoader(train_data_3, batch_size=20, num_workers=1)
testloader = torch.utils.data.DataLoader(test_data_3, batch_size=20, num_workers=1)
mlp_mine_ternary_10 = MLP_Mine_Ternary_10()
criterion = nn.CrossEntropyLoss()
# optimizer = torch.optim.Adam(mlp_g.parameters(), lr=1e-4)
optimizer = torch.optim.SGD(mlp_mine_ternary_10.parameters(), lr=0.01)

test_loss_min = 5

for epoch in range(n_epochs):
    train_loss = 0
    test_loss = 0
    
    mlp_mine_ternary_10.train()
    for data, target in trainloader:
        # Zero the gradients
        optimizer.zero_grad()

        # Perform forward pass
        output = mlp_mine_ternary_10(data.float())

        # Compute loss
        loss = criterion(output, target)

        # Perform backward pass
        loss.backward()

        # Perform optimization
        optimizer.step()

        # Print statistics
        train_loss += loss.item()*data.size(0)
        
    with torch.no_grad():
        for data, target in testloader:
            output = mlp_mine_ternary_10(data.float())
            loss = criterion(output, target)
            test_loss += loss.item()*data.size(0)
        
    train_loss = train_loss/len(trainloader.dataset)
    test_loss = test_loss/len(testloader.dataset)
    
    print('Epoch: {} \tTraining Loss: {:.6f} \tTest Loss: {:.6f}'.format(
        epoch+1, 
        train_loss,
        test_loss
        ))
    
    if test_loss <= test_loss_min:
        torch.save(mlp_mine_ternary_10.state_dict(), 'model_mine_ternary_10.pt')
        test_loss_min = test_loss

  # Process is complete.
print('All done.')

Epoch: 1 	Training Loss: 0.855436 	Test Loss: 0.832336
Epoch: 2 	Training Loss: 0.814165 	Test Loss: 0.826591
Epoch: 3 	Training Loss: 0.798119 	Test Loss: 0.825160
Epoch: 4 	Training Loss: 0.784770 	Test Loss: 0.825687
Epoch: 5 	Training Loss: 0.773858 	Test Loss: 0.831724
All done.


In [124]:
mlp_mine_ternary_10.load_state_dict(torch.load('model_mine_ternary_10.pt'))
testloader = torch.utils.data.DataLoader(test_data_3, batch_size=1, num_workers=1)

In [125]:
predictions = predict(mlp_mine_ternary_10, testloader)
predictions = np.array(predictions)

In [126]:
print("Test Accuracy:", accuracy(Y_test_3, predictions))

Test Accuracy: 0.64124


#### First 10 Features - Google Model:

In [24]:
def first_10_w2v_google(review):
    features = np.zeros((10,300))
    for i, word in enumerate(review):
        if i >= 10:
            break
        elif word in word2vec:
            features[i] = word2vec[word]
        else:
            continue
    return features  

In [128]:
# Vectorize X_train, X_test:
X_train_10 = X_train.apply(first_10_w2v_google)
X_test_10 = X_test.apply(first_10_w2v_google)

In [129]:
# Formatting the data for DataLoader (coverts to tensors):
train_data = []
for i in range(len(X_train_10)):
    train_data.append([X_train_10.iloc[i], Y_train[i]])

In [130]:
test_data = []
for i in range(len(X_test_10)):
    test_data.append([X_test_10.iloc[i], Y_test[i]])

In [131]:
class MLP_Google_Binary_10(nn.Module):

    def __init__(self):
        super().__init__()

        vocab_size = 10
        embedding_size = 300
        hidden_1 = 50
        hidden_2 = 10
        
#         self.embeddings = nn.Embedding(vocab_size, embedding_size)
        self.fc1 = nn.Linear(vocab_size*embedding_size, hidden_1)
        self.fc2 = nn.Linear(hidden_1, hidden_2)
        self.fc3 = nn.Linear(hidden_2, 2)
        self.dropout = nn.Dropout(0.2)

    def forward(self, x):
        x = x.view(-1, 3000)
        x = F.relu(self.fc1(x))
        x = self.dropout(x)
        x = F.relu(self.fc2(x))
        x = self.dropout(x)
        x = self.fc3(x)
        return x

In [132]:
n_epochs = 5
trainloader = torch.utils.data.DataLoader(train_data, batch_size=20, num_workers=1)
testloader = torch.utils.data.DataLoader(test_data, batch_size=20, num_workers=1)
mlp_google_binary_10 = MLP_Google_Binary_10()
criterion = nn.CrossEntropyLoss()
# optimizer = torch.optim.Adam(mlp_g.parameters(), lr=1e-4)
optimizer = torch.optim.SGD(mlp_google_binary_10.parameters(), lr=0.01)

test_loss_min = 5

for epoch in range(n_epochs):
    train_loss = 0
    test_loss = 0
    
    mlp_google_binary_10.train()
    for data, target in trainloader:
        # Zero the gradients
        optimizer.zero_grad()

        # Perform forward pass
        output = mlp_google_binary_10(data.float())

        # Compute loss
        loss = criterion(output, target)

        # Perform backward pass
        loss.backward()

        # Perform optimization
        optimizer.step()

        # Print statistics
        train_loss += loss.item()*data.size(0)
        
    with torch.no_grad():
        for data, target in testloader:
            output = mlp_google_binary_10(data.float())
            loss = criterion(output, target)
            test_loss += loss.item()*data.size(0)
        
    train_loss = train_loss/len(trainloader.dataset)
    test_loss = test_loss/len(testloader.dataset)
    
    print('Epoch: {} \tTraining Loss: {:.6f} \tTest Loss: {:.6f}'.format(
        epoch+1, 
        train_loss,
        test_loss
        ))
    
    if test_loss <= test_loss_min:
        torch.save(mlp_google_binary_10.state_dict(), 'model_google_binary_10.pt')
        test_loss_min = test_loss

  # Process is complete.
print('All done.')

Epoch: 1 	Training Loss: 0.579629 	Test Loss: 0.502423
Epoch: 2 	Training Loss: 0.486897 	Test Loss: 0.484351
Epoch: 3 	Training Loss: 0.466375 	Test Loss: 0.481704
Epoch: 4 	Training Loss: 0.449035 	Test Loss: 0.479321
Epoch: 5 	Training Loss: 0.431224 	Test Loss: 0.480469
All done.


In [133]:
mlp_google_binary_10.load_state_dict(torch.load('model_google_binary_10.pt'))
testloader = torch.utils.data.DataLoader(test_data, batch_size=1, num_workers=1)

In [134]:
predictions = predict(mlp_google_binary_10, testloader)
predictions = np.array(predictions)

In [135]:
print("Test Accuracy:", accuracy(Y_test, predictions))

Test Accuracy: 0.7743954338356782


In [25]:
# Vectorize X_train_3, X_test_3:
X_train_3_10 = X_train_3.apply(first_10_w2v_google)
X_test_3_10 = X_test_3.apply(first_10_w2v_google)

In [26]:
# Formatting the data for DataLoader (coverts to tensors):
train_data_3 = []
for i in range(50000):
    train_data_3.append([X_train_3_10.iloc[i], Y_train_3[i]])
    
test_data_3 = []
for i in range(len(X_test_3_10 )):
    test_data_3.append([X_test_3_10.iloc[i], Y_test_3[i]])

In [27]:
class MLP_Google_Ternary_10(nn.Module):

    def __init__(self):
        super().__init__()

        vocab_size = 10
        embedding_size = 300
        hidden_1 = 50
        hidden_2 = 10
        
#         self.embeddings = nn.Embedding(vocab_size, embedding_size)
        self.fc1 = nn.Linear(vocab_size*embedding_size, hidden_1)
        self.fc2 = nn.Linear(hidden_1, hidden_2)
        self.fc3 = nn.Linear(hidden_2, 3)
        self.dropout = nn.Dropout(0.2)

    def forward(self, x):
        x = x.view(-1, 3000)
        x = F.relu(self.fc1(x))
        x = self.dropout(x)
        x = F.relu(self.fc2(x))
        x = self.dropout(x)
        x = self.fc3(x)
        return x

In [28]:
n_epochs = 5
trainloader = torch.utils.data.DataLoader(train_data_3, batch_size=20, num_workers=1)
testloader = torch.utils.data.DataLoader(test_data_3, batch_size=20, num_workers=1)
mlp_google_ternary_10 = MLP_Google_Ternary_10()
criterion = nn.CrossEntropyLoss()
# optimizer = torch.optim.Adam(mlp_g.parameters(), lr=1e-4)
optimizer = torch.optim.SGD(mlp_google_ternary_10.parameters(), lr=0.01)

test_loss_min = 5

for epoch in range(n_epochs):
    train_loss = 0
    test_loss = 0
    
    mlp_google_ternary_10.train()
    for data, target in trainloader:
        # Zero the gradients
        optimizer.zero_grad()

        # Perform forward pass
        output = mlp_google_ternary_10(data.float())

        # Compute loss
        loss = criterion(output, target)

        # Perform backward pass
        loss.backward()

        # Perform optimization
        optimizer.step()

        # Print statistics
        train_loss += loss.item()*data.size(0)
        
    with torch.no_grad():
        for data, target in testloader:
            output = mlp_google_ternary_10(data.float())
            loss = criterion(output, target)
            test_loss += loss.item()*data.size(0)
        
    train_loss = train_loss/len(trainloader.dataset)
    test_loss = test_loss/len(testloader.dataset)
    
    print('Epoch: {} \tTraining Loss: {:.6f} \tTest Loss: {:.6f}'.format(
        epoch+1, 
        train_loss,
        test_loss
        ))
    
    if test_loss <= test_loss_min:
        torch.save(mlp_google_ternary_10.state_dict(), 'model_google_ternary_10.pt')
        test_loss_min = test_loss

  # Process is complete.
print('All done.')

Epoch: 1 	Training Loss: 1.027988 	Test Loss: 0.965437
Epoch: 2 	Training Loss: 0.925587 	Test Loss: 0.918224
Epoch: 3 	Training Loss: 0.888207 	Test Loss: 0.901832
Epoch: 4 	Training Loss: 0.865427 	Test Loss: 0.896150
Epoch: 5 	Training Loss: 0.844262 	Test Loss: 0.891915
All done.


In [29]:
mlp_google_ternary_10.load_state_dict(torch.load('model_google_ternary_10.pt'))
testloader = torch.utils.data.DataLoader(test_data_3, batch_size=1, num_workers=1)

In [30]:
predictions = predict(mlp_google_ternary_10, testloader)
predictions = np.array(predictions)

In [31]:
print("Test Accuracy:", accuracy(Y_test_3, predictions))

Test Accuracy: 0.60056


### 5. RNN

In [43]:
def first_50_w2v_mine(review):
    features = np.zeros((10,200))
    for i, word in enumerate(review):
        if i >= 10:
            break
        elif word in model.wv:
            features[i] = model.wv[word]
        else:
            continue
    return features 

In [33]:
# Vectorize X_train, X_test:
X_train_50 = X_train.apply(first_50_w2v_mine)
X_test_50 = X_test.apply(first_50_w2v_mine)

In [34]:
# Formatting the data for DataLoader (coverts to tensors):
train_data = []
for i in range(len(X_train_50)):
    train_data.append([X_train_50.iloc[i], Y_train[i]])

In [35]:
test_data = []
for i in range(len(X_test_50)):
    test_data.append([X_test_50.iloc[i], Y_test[i]])

In [36]:
from torch.autograd import Variable
class RNN_Mine_Binary_50(nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim):
        super(RNN_Mine_Binary_50, self).__init__()
        
        self.input_dim = input_dim
        self.hidden_dim = hidden_dim
        self.output_dim = output_dim
        self.layer_dim = 1

        self.rnn = nn.RNN(self.input_dim, self.hidden_dim, self.layer_dim, batch_first=True, nonlinearity='relu')
        self.fc = nn.Linear(self.hidden_dim, self.output_dim)

    def forward(self, x):
        # Initialize hidden state with zeros
        h0 = Variable(torch.zeros(self.layer_dim, x.size(0), self.hidden_dim))
            
        # One time step
        out, hn = self.rnn(x, h0)
        out = self.fc(out[:, -1, :]) 
        out = nn.Softmax(dim=1)(out)
        return out

In [37]:
n_epochs = 5
trainloader = torch.utils.data.DataLoader(train_data, batch_size=20, num_workers=1)
testloader = torch.utils.data.DataLoader(test_data, batch_size=20, num_workers=1)
rnn_mine_binary_50 = RNN_Mine_Binary_50(200,50,2)
criterion = nn.CrossEntropyLoss()
# optimizer = torch.optim.Adam(mlp_g.parameters(), lr=1e-4)
optimizer = torch.optim.SGD(rnn_mine_binary_50.parameters(), lr=0.01)

test_loss_min = 5

for epoch in range(n_epochs):
    train_loss = 0
    test_loss = 0
    
    rnn_mine_binary_50.train()
    for data, target in trainloader:
        # Zero the gradients
        optimizer.zero_grad()

        # Perform forward pass
        output = rnn_mine_binary_50(data.float())

        # Compute loss
        loss = criterion(output, target)

        # Perform backward pass
        loss.backward()

        # Perform optimization
        optimizer.step()

        # Print statistics
        train_loss += loss.item()*data.size(0)
        
    with torch.no_grad():
        for data, target in testloader:
            output = rnn_mine_binary_50(data.float())
            loss = criterion(output, target)
            test_loss += loss.item()*data.size(0)
        
    train_loss = train_loss/len(trainloader.dataset)
    test_loss = test_loss/len(testloader.dataset)
    
    print('Epoch: {} \tTraining Loss: {:.6f} \tTest Loss: {:.6f}'.format(
        epoch+1, 
        train_loss,
        test_loss
        ))
    
    if test_loss <= test_loss_min:
        torch.save(rnn_mine_binary_50.state_dict(), 'rnn_mine_binary_50.pt')
        test_loss_min = test_loss

  # Process is complete.
print('All done.')

Epoch: 1 	Training Loss: 0.548213 	Test Loss: 0.514838
Epoch: 2 	Training Loss: 0.505670 	Test Loss: 0.510650
Epoch: 3 	Training Loss: 0.500859 	Test Loss: 0.506438
Epoch: 4 	Training Loss: 0.497493 	Test Loss: 0.502784
Epoch: 5 	Training Loss: 0.494619 	Test Loss: 0.502303
All done.


In [38]:
rnn_mine_binary_50.load_state_dict(torch.load('rnn_mine_binary_50.pt'))
testloader = torch.utils.data.DataLoader(test_data, batch_size=1, num_workers=1)

In [39]:
predictions = predict(rnn_mine_binary_50, testloader)
predictions = np.array(predictions)

In [40]:
print("Test Accuracy:", accuracy(Y_test, predictions))

Test Accuracy: 0.8009377664109122


In [41]:
# Vectorize X_train_3, X_test_3:
X_train_3_50 = X_train_3.apply(first_50_w2v_mine)
X_test_3_50 = X_test_3.apply(first_50_w2v_mine)

In [42]:
# Formatting the data for DataLoader (coverts to tensors):
train_data_3 = []
for i in range(len(X_train_3_50)):
    train_data_3.append([X_train_3_50.iloc[i], Y_train_3[i]])
    
test_data_3 = []
for i in range(len(X_test_3_50)):
    test_data_3.append([X_test_3_50.iloc[i], Y_test_3[i]])

In [43]:
class RNN_Mine_Ternary_50(nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim):
        super(RNN_Mine_Ternary_50, self).__init__()
        
        self.input_dim = input_dim
        self.hidden_dim = hidden_dim
        self.output_dim = output_dim
        self.layer_dim = 1

        self.rnn = nn.RNN(self.input_dim, self.hidden_dim, self.layer_dim, batch_first=True, nonlinearity='relu')
        self.fc = nn.Linear(self.hidden_dim, self.output_dim)

    def forward(self, x):
        # Initialize hidden state with zeros
        h0 = Variable(torch.zeros(self.layer_dim, x.size(0), self.hidden_dim))
            
        # One time step
        out, hn = self.rnn(x, h0)
        out = self.fc(out[:, -1, :]) 
        out = nn.Softmax(dim=1)(out)
        return out

In [44]:
n_epochs = 5
trainloader = torch.utils.data.DataLoader(train_data_3, batch_size=20, num_workers=1)
testloader = torch.utils.data.DataLoader(test_data_3, batch_size=20, num_workers=1)
rnn_mine_ternary_50 = RNN_Mine_Ternary_50(200,50,3)
criterion = nn.CrossEntropyLoss()
# optimizer = torch.optim.Adam(mlp_g.parameters(), lr=1e-4)
optimizer = torch.optim.SGD(rnn_mine_ternary_50.parameters(), lr=0.01)

test_loss_min = 5

for epoch in range(n_epochs):
    train_loss = 0
    test_loss = 0
    
    rnn_mine_ternary_50.train()
    for data, target in trainloader:
        # Zero the gradients
        optimizer.zero_grad()

        # Perform forward pass
        output = rnn_mine_ternary_50(data.float())

        # Compute loss
        loss = criterion(output, target)

        # Perform backward pass
        loss.backward()

        # Perform optimization
        optimizer.step()

        # Print statistics
        train_loss += loss.item()*data.size(0)
        
    with torch.no_grad():
        for data, target in testloader:
            output = rnn_mine_ternary_50(data.float())
            loss = criterion(output, target)
            test_loss += loss.item()*data.size(0)
        
    train_loss = train_loss/len(trainloader.dataset)
    test_loss = test_loss/len(testloader.dataset)
    
    print('Epoch: {} \tTraining Loss: {:.6f} \tTest Loss: {:.6f}'.format(
        epoch+1, 
        train_loss,
        test_loss
        ))
    
    if test_loss <= test_loss_min:
        torch.save(rnn_mine_ternary_50.state_dict(), 'rnn_mine_ternary_50.pt')
        test_loss_min = test_loss

  # Process is complete.
print('All done.')

Epoch: 1 	Training Loss: 0.935764 	Test Loss: 0.908988
Epoch: 2 	Training Loss: 0.900973 	Test Loss: 0.906607
Epoch: 3 	Training Loss: 0.896076 	Test Loss: 0.902012
Epoch: 4 	Training Loss: 0.892633 	Test Loss: 0.900076
Epoch: 5 	Training Loss: 0.889609 	Test Loss: 0.897427
All done.


In [45]:
rnn_mine_ternary_50.load_state_dict(torch.load('rnn_mine_ternary_50.pt'))
testloader = torch.utils.data.DataLoader(test_data_3, batch_size=1, num_workers=1)

In [46]:
predictions = predict(rnn_mine_ternary_50, testloader)
predictions = np.array(predictions)

In [47]:
print("Test Accuracy:", accuracy(Y_test_3, predictions))

Test Accuracy: 0.64018


In [1]:
def first_20_w2v_google(review):
    features = np.zeros((20,300))
    for i, word in enumerate(review):
        if i >= 10:
            break
        elif word in word2vec:
            features[i] = word2vec[word]
        else:
            continue
    return features 

In [25]:
# Vectorize X_train, X_test:
X_train_20 = X_train.apply(first_20_w2v_google)
X_test_20 = X_test.apply(first_20_w2v_google)

In [26]:
# Formatting the data for DataLoader (coverts to tensors):
train_data = []
for i in range(len(X_train_20)):
    train_data.append([X_train_20.iloc[i], Y_train[i]])

In [27]:
test_data = []
for i in range(len(X_test_20)):
    test_data.append([X_test_20.iloc[i], Y_test[i]])

In [28]:
from torch.autograd import Variable
class RNN_Google_Binary_20(nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim):
        super(RNN_Google_Binary_20, self).__init__()
        
        self.input_dim = input_dim
        self.hidden_dim = hidden_dim
        self.output_dim = output_dim
        self.layer_dim = 1

        self.rnn = nn.RNN(self.input_dim, self.hidden_dim, self.layer_dim, batch_first=True, nonlinearity='relu')
        self.fc = nn.Linear(self.hidden_dim, self.output_dim)

    def forward(self, x):
        # Initialize hidden state with zeros
        h0 = Variable(torch.zeros(self.layer_dim, x.size(0), self.hidden_dim))
            
        # One time step
        out, hn = self.rnn(x, h0)
        out = self.fc(out[:, -1, :]) 
        out = nn.Softmax(dim=1)(out)
        return out

In [29]:
n_epochs = 5
trainloader = torch.utils.data.DataLoader(train_data, batch_size=20, num_workers=1)
testloader = torch.utils.data.DataLoader(test_data, batch_size=20, num_workers=1)
rnn_google_binary_20 = RNN_Google_Binary_20(300,50,2)
# rnn_google_binary_50.cuda()
# device = torch.device("cuda")
criterion = nn.CrossEntropyLoss()
# optimizer = torch.optim.Adam(mlp_g.parameters(), lr=1e-4)
optimizer = torch.optim.SGD(rnn_google_binary_20.parameters(), lr=0.01)

test_loss_min = 5

for epoch in range(n_epochs):
    train_loss = 0
    test_loss = 0
    
    rnn_google_binary_20.train()
    for data, target in trainloader:
        # Zero the gradients
        optimizer.zero_grad()

        # Perform forward pass
        output = rnn_google_binary_20(data.float())

        # Compute loss
        loss = criterion(output, target)

        # Perform backward pass
        loss.backward()

        # Perform optimization
        optimizer.step()

        # Print statistics
        train_loss += loss.item()*data.size(0)
        
    with torch.no_grad():
        for data, target in testloader:
            output = rnn_google_binary_20(data.float())
            loss = criterion(output, target)
            test_loss += loss.item()*data.size(0)
        
    train_loss = train_loss/len(trainloader.dataset)
    test_loss = test_loss/len(testloader.dataset)
    
    print('Epoch: {} \tTraining Loss: {:.6f} \tTest Loss: {:.6f}'.format(
        epoch+1, 
        train_loss,
        test_loss
        ))
    
    if test_loss <= test_loss_min:
        torch.save(rnn_google_binary_20.state_dict(), 'rnn_google_binary_20.pt')
        test_loss_min = test_loss

  # Process is complete.
print('All done.')

Epoch: 1 	Training Loss: 0.693192 	Test Loss: 0.693167
Epoch: 2 	Training Loss: 0.693178 	Test Loss: 0.693166
Epoch: 3 	Training Loss: 0.693178 	Test Loss: 0.693167
Epoch: 4 	Training Loss: 0.693178 	Test Loss: 0.693167
Epoch: 5 	Training Loss: 0.693177 	Test Loss: 0.693167
All done.


In [30]:
rnn_google_binary_20.load_state_dict(torch.load('rnn_google_binary_20.pt'))
testloader = torch.utils.data.DataLoader(test_data, batch_size=1, num_workers=1)

In [31]:
predictions = predict(rnn_google_binary_20, testloader)
predictions = np.array(predictions)

In [33]:
print("Test Accuracy:", accuracy(Y_test, predictions))

Test Accuracy: 0.49921184977606525


In [33]:
def first_10_w2v_google(review):
    features = np.zeros((10,300))
    for i, word in enumerate(review):
        if i >= 10:
            break
        elif word in word2vec:
            features[i] = word2vec[word]
        else:
            continue
    return features 

In [35]:
# Vectorize X_train_3, X_test_3:
X_train_3_10 = X_train_3.apply(first_10_w2v_google)
X_test_3_10 = X_test_3.apply(first_10_w2v_google)

In [36]:
# Formatting the data for DataLoader (coverts to tensors):
train_data_3 = []
for i in range(len(X_train_3_10)):
    train_data_3.append([X_train_3_10.iloc[i], Y_train_3[i]])
    
test_data_3 = []
for i in range(len(X_test_3_10)):
    test_data_3.append([X_test_3_10.iloc[i], Y_test_3[i]])

In [37]:
from torch.autograd import Variable
class RNN_Google_Ternary_10(nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim):
        super(RNN_Google_Ternary_10, self).__init__()
        
        self.input_dim = input_dim
        self.hidden_dim = hidden_dim
        self.output_dim = output_dim
        self.layer_dim = 1

        self.rnn = nn.RNN(self.input_dim, self.hidden_dim, self.layer_dim, batch_first=True, nonlinearity='relu')
        self.fc = nn.Linear(self.hidden_dim, self.output_dim)

    def forward(self, x):
        # Initialize hidden state with zeros
        h0 = Variable(torch.zeros(self.layer_dim, x.size(0), self.hidden_dim))
            
        # One time step
        out, hn = self.rnn(x, h0)
        out = self.fc(out[:, -1, :]) 
        out = nn.Softmax(dim=1)(out)
        return out

In [38]:
n_epochs = 5
trainloader = torch.utils.data.DataLoader(train_data_3, batch_size=20, num_workers=1)
testloader = torch.utils.data.DataLoader(test_data_3, batch_size=20, num_workers=1)
rnn_google_ternary_10 = RNN_Google_Ternary_10(300,50,3)
# rnn_google_binary_50.cuda()
# device = torch.device("cuda")
criterion = nn.CrossEntropyLoss()
# optimizer = torch.optim.Adam(mlp_g.parameters(), lr=1e-4)
optimizer = torch.optim.SGD(rnn_google_ternary_10.parameters(), lr=0.01)

test_loss_min = 5

for epoch in range(n_epochs):
    train_loss = 0
    test_loss = 0
    
    rnn_google_ternary_10.train()
    for data, target in trainloader:
        # Zero the gradients
        optimizer.zero_grad()

        # Perform forward pass
        output = rnn_google_ternary_10(data.float())

        # Compute loss
        loss = criterion(output, target)

        # Perform backward pass
        loss.backward()

        # Perform optimization
        optimizer.step()

        # Print statistics
        train_loss += loss.item()*data.size(0)
        
    with torch.no_grad():
        for data, target in testloader:
            output = rnn_google_ternary_10(data.float())
            loss = criterion(output, target)
            test_loss += loss.item()*data.size(0)
        
    train_loss = train_loss/len(trainloader.dataset)
    test_loss = test_loss/len(testloader.dataset)
    
    print('Epoch: {} \tTraining Loss: {:.6f} \tTest Loss: {:.6f}'.format(
        epoch+1, 
        train_loss,
        test_loss
        ))
    
    if test_loss <= test_loss_min:
        torch.save(rnn_google_ternary_10.state_dict(), 'rnn_google_ternary_10.pt')
        test_loss_min = test_loss

  # Process is complete.
print('All done.')

Epoch: 1 	Training Loss: 1.018233 	Test Loss: 0.946186
Epoch: 2 	Training Loss: 0.935987 	Test Loss: 0.930667
Epoch: 3 	Training Loss: 0.924268 	Test Loss: 0.922230
Epoch: 4 	Training Loss: 0.916954 	Test Loss: 0.915665
Epoch: 5 	Training Loss: 0.911168 	Test Loss: 0.910946
All done.


In [39]:
rnn_google_ternary_10.load_state_dict(torch.load('rnn_google_ternary_10.pt'))
testloader = torch.utils.data.DataLoader(test_data_3, batch_size=1, num_workers=1)

In [40]:
predictions = predict(rnn_google_ternary_10, testloader)
predictions = np.array(predictions)

In [41]:
print("Test Accuracy:", accuracy(Y_test_3, predictions))

Test Accuracy: 0.62742


### GRU

In [44]:
# Vectorize X_train, X_test:
X_train_50 = X_train.apply(first_50_w2v_mine)
X_test_50 = X_test.apply(first_50_w2v_mine)

In [45]:
# Formatting the data for DataLoader (coverts to tensors):
train_data = []
for i in range(len(X_train_50)):
    train_data.append([X_train_50.iloc[i], Y_train[i]])

In [46]:
test_data = []
for i in range(len(X_test_50)):
    test_data.append([X_test_50.iloc[i], Y_test[i]])

In [47]:
from torch.autograd import Variable
class GRU_Mine_Binary_50(nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim):
        super(GRU_Mine_Binary_50, self).__init__()
        
        self.input_dim = input_dim
        self.hidden_dim = hidden_dim
        self.output_dim = output_dim
        self.layer_dim = 1

        self.rnn = nn.GRU(self.input_dim, self.hidden_dim, self.layer_dim, batch_first=True)
        self.fc = nn.Linear(self.hidden_dim, self.output_dim)

    def forward(self, x):
        # Initialize hidden state with zeros
        h0 = Variable(torch.zeros(self.layer_dim, x.size(0), self.hidden_dim))
            
        # One time step
        out, hn = self.rnn(x, h0)
        out = self.fc(out[:, -1, :]) 
        out = nn.Softmax(dim=1)(out)
        return out

In [48]:
n_epochs = 5
trainloader = torch.utils.data.DataLoader(train_data, batch_size=20, num_workers=1)
testloader = torch.utils.data.DataLoader(test_data, batch_size=20, num_workers=1)
gru_mine_binary_50 = GRU_Mine_Binary_50(200,50,2)
criterion = nn.CrossEntropyLoss()
# optimizer = torch.optim.Adam(mlp_g.parameters(), lr=1e-4)
optimizer = torch.optim.SGD(gru_mine_binary_50.parameters(), lr=0.01)

test_loss_min = 5

for epoch in range(n_epochs):
    train_loss = 0
    test_loss = 0
    
    gru_mine_binary_50.train()
    for data, target in trainloader:
        # Zero the gradients
        optimizer.zero_grad()

        # Perform forward pass
        output = gru_mine_binary_50(data.float())

        # Compute loss
        loss = criterion(output, target)

        # Perform backward pass
        loss.backward()

        # Perform optimization
        optimizer.step()

        # Print statistics
        train_loss += loss.item()*data.size(0)
        
    with torch.no_grad():
        for data, target in testloader:
            output = gru_mine_binary_50(data.float())
            loss = criterion(output, target)
            test_loss += loss.item()*data.size(0)
        
    train_loss = train_loss/len(trainloader.dataset)
    test_loss = test_loss/len(testloader.dataset)
    
    print('Epoch: {} \tTraining Loss: {:.6f} \tTest Loss: {:.6f}'.format(
        epoch+1, 
        train_loss,
        test_loss
        ))
    
    if test_loss <= test_loss_min:
        torch.save(gru_mine_binary_50.state_dict(), 'gru_mine_binary_50.pt')
        test_loss_min = test_loss

  # Process is complete.
print('All done.')

Epoch: 1 	Training Loss: 0.588531 	Test Loss: 0.522622
Epoch: 2 	Training Loss: 0.508419 	Test Loss: 0.502364
Epoch: 3 	Training Loss: 0.497792 	Test Loss: 0.498301
Epoch: 4 	Training Loss: 0.492826 	Test Loss: 0.496207
Epoch: 5 	Training Loss: 0.489045 	Test Loss: 0.494325
All done.


In [49]:
gru_mine_binary_50.load_state_dict(torch.load('gru_mine_binary_50.pt'))
testloader = torch.utils.data.DataLoader(test_data, batch_size=1, num_workers=1)

In [50]:
predictions = predict(gru_mine_binary_50, testloader)
predictions = np.array(predictions)

In [51]:
print("Test Accuracy:", accuracy(Y_test, predictions))

Test Accuracy: 0.8086421297570495


In [52]:
# Vectorize X_train_3, X_test_3:
X_train_3_50 = X_train_3.apply(first_50_w2v_mine)
X_test_3_50 = X_test_3.apply(first_50_w2v_mine)

In [53]:
# Formatting the data for DataLoader (coverts to tensors):
train_data_3 = []
for i in range(len(X_train_3_50)):
    train_data_3.append([X_train_3_50.iloc[i], Y_train_3[i]])
    
test_data_3 = []
for i in range(len(X_test_3_50)):
    test_data_3.append([X_test_3_50.iloc[i], Y_test_3[i]])

In [54]:
from torch.autograd import Variable
class GRU_Mine_Ternary_50(nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim):
        super(GRU_Mine_Ternary_50, self).__init__()
        
        self.input_dim = input_dim
        self.hidden_dim = hidden_dim
        self.output_dim = output_dim
        self.layer_dim = 1

        self.rnn = nn.GRU(self.input_dim, self.hidden_dim, self.layer_dim, batch_first=True)
        self.fc = nn.Linear(self.hidden_dim, self.output_dim)

    def forward(self, x):
        # Initialize hidden state with zeros
        h0 = Variable(torch.zeros(self.layer_dim, x.size(0), self.hidden_dim))
            
        # One time step
        out, hn = self.rnn(x, h0)
        out = self.fc(out[:, -1, :]) 
        out = nn.Softmax(dim=1)(out)
        return out

In [55]:
n_epochs = 5
trainloader = torch.utils.data.DataLoader(train_data_3, batch_size=20, num_workers=1)
testloader = torch.utils.data.DataLoader(test_data_3, batch_size=20, num_workers=1)
gru_mine_ternary_50 = GRU_Mine_Ternary_50(200,50,3)
# rnn_google_binary_50.cuda()
# device = torch.device("cuda")
criterion = nn.CrossEntropyLoss()
# optimizer = torch.optim.Adam(mlp_g.parameters(), lr=1e-4)
optimizer = torch.optim.SGD(gru_mine_ternary_50.parameters(), lr=0.01)

test_loss_min = 5

for epoch in range(n_epochs):
    train_loss = 0
    test_loss = 0
    
    gru_mine_ternary_50.train()
    for data, target in trainloader:
        # Zero the gradients
        optimizer.zero_grad()

        # Perform forward pass
        output = gru_mine_ternary_50(data.float())

        # Compute loss
        loss = criterion(output, target)

        # Perform backward pass
        loss.backward()

        # Perform optimization
        optimizer.step()

        # Print statistics
        train_loss += loss.item()*data.size(0)
        
    with torch.no_grad():
        for data, target in testloader:
            output = gru_mine_ternary_50(data.float())
            loss = criterion(output, target)
            test_loss += loss.item()*data.size(0)
        
    train_loss = train_loss/len(trainloader.dataset)
    test_loss = test_loss/len(testloader.dataset)
    
    print('Epoch: {} \tTraining Loss: {:.6f} \tTest Loss: {:.6f}'.format(
        epoch+1, 
        train_loss,
        test_loss
        ))
    
    if test_loss <= test_loss_min:
        torch.save(gru_mine_ternary_50.state_dict(), 'gru_mine_ternary_50.pt')
        test_loss_min = test_loss

  # Process is complete.
print('All done.')

Epoch: 1 	Training Loss: 0.973553 	Test Loss: 0.916444
Epoch: 2 	Training Loss: 0.905931 	Test Loss: 0.901182
Epoch: 3 	Training Loss: 0.896875 	Test Loss: 0.896730
Epoch: 4 	Training Loss: 0.891677 	Test Loss: 0.893707
Epoch: 5 	Training Loss: 0.887691 	Test Loss: 0.890985
All done.


In [56]:
gru_mine_ternary_50.load_state_dict(torch.load('gru_mine_ternary_50.pt'))
testloader = torch.utils.data.DataLoader(test_data_3, batch_size=1, num_workers=1)

In [57]:
predictions = predict(gru_mine_ternary_50, testloader)
predictions = np.array(predictions)

In [58]:
print("Test Accuracy:", accuracy(Y_test_3, predictions))

Test Accuracy: 0.6485


#### GRU-Google:

In [24]:
# Vectorize X_train, X_test:
X_train_50 = X_train.apply(first_20_w2v_google)
X_test_50 = X_test.apply(first_20_w2v_google)

In [25]:
# Formatting the data for DataLoader (coverts to tensors):
train_data = []
for i in range(50000):
    train_data.append([X_train_50.iloc[i], Y_train[i]])

In [26]:
test_data = []
for i in range(len(X_test_50)):
    test_data.append([X_test_50.iloc[i], Y_test[i]])

In [27]:
from torch.autograd import Variable
class GRU_Google_Binary_50(nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim):
        super(GRU_Google_Binary_50, self).__init__()
        
        self.input_dim = input_dim
        self.hidden_dim = hidden_dim
        self.output_dim = output_dim
        self.layer_dim = 1

        self.gru = nn.GRU(self.input_dim, self.hidden_dim, self.layer_dim, batch_first=True)
        self.fc = nn.Linear(self.hidden_dim, self.output_dim)

    def forward(self, x):
        # Initialize hidden state with zeros
        h0 = Variable(torch.zeros(self.layer_dim, x.size(0), self.hidden_dim))
            
        # One time step
        out, hn = self.gru(x, h0)
        out = self.fc(out[:, -1, :]) 
        out = nn.Softmax(dim=1)(out)
        return out

In [28]:
n_epochs = 5
trainloader = torch.utils.data.DataLoader(train_data, batch_size=20, num_workers=1)
testloader = torch.utils.data.DataLoader(test_data, batch_size=20, num_workers=1)
gru_google_binary_50 = GRU_Google_Binary_50(300,50,2)
criterion = nn.CrossEntropyLoss()
# optimizer = torch.optim.Adam(mlp_g.parameters(), lr=1e-4)
optimizer = torch.optim.SGD(gru_google_binary_50.parameters(), lr=0.1)

test_loss_min = 5

for epoch in range(n_epochs):
    train_loss = 0
    test_loss = 0
    
    gru_google_binary_50.train()
    for data, target in trainloader:
        # Zero the gradients
        optimizer.zero_grad()

        # Perform forward pass
        output = gru_google_binary_50(data.float())

        # Compute loss
        loss = criterion(output, target)

        # Perform backward pass
        loss.backward()

        # Perform optimization
        optimizer.step()

        # Print statistics
        train_loss += loss.item()*data.size(0)
        
    with torch.no_grad():
        for data, target in testloader:
            output = gru_google_binary_50(data.float())
            loss = criterion(output, target)
            test_loss += loss.item()*data.size(0)
        
    train_loss = train_loss/len(trainloader.dataset)
    test_loss = test_loss/len(testloader.dataset)
    
    print('Epoch: {} \tTraining Loss: {:.6f} \tTest Loss: {:.6f}'.format(
        epoch+1, 
        train_loss,
        test_loss
        ))
    
    if test_loss <= test_loss_min:
        torch.save(gru_google_binary_50.state_dict(), 'gru_google_binary_50.pt')
        test_loss_min = test_loss

  # Process is complete.
print('All done.')

Epoch: 1 	Training Loss: 0.693459 	Test Loss: 0.693150
Epoch: 2 	Training Loss: 0.693412 	Test Loss: 0.693142
Epoch: 3 	Training Loss: 0.693386 	Test Loss: 0.693136
Epoch: 4 	Training Loss: 0.693366 	Test Loss: 0.693131
Epoch: 5 	Training Loss: 0.693350 	Test Loss: 0.693125
All done.


In [29]:
gru_google_binary_50.load_state_dict(torch.load('gru_google_binary_50.pt'))
testloader = torch.utils.data.DataLoader(test_data, batch_size=1, num_workers=1)

In [30]:
predictions = predict(gru_google_binary_50, testloader)
predictions = np.array(predictions)

In [31]:
print("Test Accuracy:", accuracy(Y_test, predictions))

Test Accuracy: 0.5809893550407013


In [34]:
# Vectorize X_train_3, X_test_3:
X_train_3_50 = X_train_3.apply(first_10_w2v_google)
X_test_3_50 = X_test_3.apply(first_10_w2v_google)

In [35]:
# Formatting the data for DataLoader (coverts to tensors):
train_data_3 = []
for i in range(50000):
    train_data_3.append([X_train_3_50.iloc[i], Y_train_3[i]])
    
test_data_3 = []
for i in range(len(X_test_3_50)):
    test_data_3.append([X_test_3_50.iloc[i], Y_test_3[i]])

In [36]:
from torch.autograd import Variable
class GRU_Google_Ternary_50(nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim):
        super(GRU_Google_Ternary_50, self).__init__()
        
        self.input_dim = input_dim
        self.hidden_dim = hidden_dim
        self.output_dim = output_dim
        self.layer_dim = 1

        self.rnn = nn.GRU(self.input_dim, self.hidden_dim, self.layer_dim, batch_first=True)
        self.fc = nn.Linear(self.hidden_dim, self.output_dim)

    def forward(self, x):
        # Initialize hidden state with zeros
        h0 = Variable(torch.zeros(self.layer_dim, x.size(0), self.hidden_dim))
            
        # One time step
        out, hn = self.rnn(x, h0)
        out = self.fc(out[:, -1, :]) 
        out = nn.Softmax(dim=1)(out)
        return out

In [ ]:
n_epochs = 5
trainloader = torch.utils.data.DataLoader(train_data_3, batch_size=20, num_workers=1)
testloader = torch.utils.data.DataLoader(test_data_3, batch_size=20, num_workers=1)
gru_google_ternary_50 = GRU_Google_Ternary_50(300,50,3)
# rnn_google_binary_50.cuda()
# device = torch.device("cuda")
criterion = nn.CrossEntropyLoss()
# optimizer = torch.optim.Adam(mlp_g.parameters(), lr=1e-4)
optimizer = torch.optim.SGD(gru_google_ternary_50.parameters(), lr=0.01)

test_loss_min = 5

for epoch in range(n_epochs):
    train_loss = 0
    test_loss = 0
    
    gru_google_ternary_50.train()
    for data, target in trainloader:
        # Zero the gradients
        optimizer.zero_grad()

        # Perform forward pass
        output = gru_google_ternary_50(data.float())

        # Compute loss
        loss = criterion(output, target)

        # Perform backward pass
        loss.backward()

        # Perform optimization
        optimizer.step()

        # Print statistics
        train_loss += loss.item()*data.size(0)
        
    with torch.no_grad():
        for data, target in testloader:
            output = gru_google_ternary_50(data.float())
            loss = criterion(output, target)
            test_loss += loss.item()*data.size(0)
        
    train_loss = train_loss/len(trainloader.dataset)
    test_loss = test_loss/len(testloader.dataset)
    
    print('Epoch: {} \tTraining Loss: {:.6f} \tTest Loss: {:.6f}'.format(
        epoch+1, 
        train_loss,
        test_loss
        ))
    
    if test_loss <= test_loss_min:
        torch.save(gru_google_ternary_50.state_dict(), 'gru_google_ternary_50.pt')
        test_loss_min = test_loss

  # Process is complete.
print('All done.')

Epoch: 1 	Training Loss: 1.071897 	Test Loss: 1.061849
Epoch: 2 	Training Loss: 1.056165 	Test Loss: 1.054676
Epoch: 3 	Training Loss: 1.048200 	Test Loss: 1.044948
Epoch: 4 	Training Loss: 1.032639 	Test Loss: 1.019699


In [ ]:
gru_google_ternary_50.load_state_dict(torch.load('gru_google_ternary_50.pt'))
testloader = torch.utils.data.DataLoader(test_data_3, batch_size=1, num_workers=1)

In [ ]:
predictions = predict(gru_google_ternary_50, testloader)
predictions = np.array(predictions)

In [ ]:
print("Test Accuracy:", accuracy(Y_test_3, predictions))